In [ ]:
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
# filtermasken generator
from math import ceil

xtrans="""gbggrg
rgrbgb
gbggrg
grggbg
bgbrgr
grggbg
"""
bayer = """gr
bg"""

def filtermask_from_str(w,h,pattern=bayer):
    d = {"r":[1,0,0],"g":[0,1,0],"b":[0,0,1]}
    a = np.array([[d[y] for y in x] for x in pattern.splitlines()]).astype("float32")
    (ph,pw,_) = a.shape
    return np.tile(a,(ceil(h/ph),ceil(w/pw),1))[0:h,0:w]

## Trainingsdaten vorbereiten
Bilder zunächst auf 50% runterskalieren um mögliche Mängel des Demosaic Verfahrens der Kamera zu verstecken. Dann wird das Bild nachträglich in eine Form gebracht die einem Bayer Sensor entspricht vor dem Demosaic Verfahren.

In [ ]:
from pathlib import Path
from PIL import Image
import numpy as np
from itertools import product
from random import seed,shuffle

tile_size=32

training_dataset=[]

for p in Path("./training_images").iterdir():
    if p.is_file():
        # bilder laden und in numpy array umwandeln
        im = Image.open(p).convert('RGB')
        # bild runterskalieren um effekte vom ursprünglichen demosaic verfahren zu minimieren
        im = im.resize((im.width//2,im.height//2),Image.LANCZOS)
        
        im_a = np.asarray(im,'float32')
        im_a/=255
        im_b = im_a.copy()
        
        # Bayer Pattern erzeugen
        im_a*=filtermask_from_str(im.width,im.height)
                
        #bild aufteilen 
        (h,w,_) = im_a.shape
        (h,w) = (h//tile_size,w//tile_size)
        
        for x,y in product(range(w),range(h)):
            tile_a = im_a[y*tile_size:(y+1)*tile_size, x*tile_size:(x+1)*tile_size,].copy()
            tile_b = im_b[y*tile_size:(y+1)*tile_size, x*tile_size:(x+1)*tile_size,].copy()
            training_dataset.append((tile_a,tile_b))

seed(42)
shuffle(training_dataset)

train_a,train_b = zip(*training_dataset)

train_a = np.array(train_a)
train_b = np.array(train_b)

### Mosaic zur Kontrolle anzeigen

In [ ]:
idx = 200

plt.figure(figsize=(16,16*2))
plt.subplot(1,2,1)
plt.imshow(train_a[idx],interpolation="nearest")
plt.text(16,-2,"Bayer Pattern",fontsize=20,horizontalalignment='center')
plt.subplot(1,2,2)
plt.imshow(train_b[idx],interpolation="nearest")
plt.text(16,-2,"RGB Image",fontsize=20,horizontalalignment='center')

plt.show()

## Modell erstellen

### Tensorflow soll sich nur den Speicher holen den es braucht anstatt sich alles unter den Nagel zu reißen
([Code aus Tensorflow Guide](https://www.tensorflow.org/guide/gpu#limiting_gpu_memory_growth))

In [ ]:
import keras
from keras import backend as K

if K.backend() == 'tensorflow':
    import tensorflow as tf
    gpus = tf.config.experimental.list_physical_devices('GPU')
    if gpus:
      try:
        # Currently, memory growth needs to be the same across GPUs
        for gpu in gpus:
          tf.config.experimental.set_memory_growth(gpu, True)
        logical_gpus = tf.config.experimental.list_logical_devices('GPU')
        print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPUs")
      except RuntimeError as e:
        # Memory growth must be set before GPUs have been initialized
        print(e)

 Hatte PSNRLoss in einem anderem Projekt benutzt und war besser als die builtin metrics

In [ ]:
def PSNRLoss(y_true, y_pred):
    """
    PSNR is Peek Signal to Noise Ratio, which is similar to mean squared error.
    It can be calculated as
    PSNR = 20 * log10(MAXp) - 10 * log10(MSE)
    When providing an unscaled input, MAXp = 255. Therefore 20 * log10(255)== 48.1308036087.
    However, since we are scaling our input, MAXp = 1. Therefore 20 * log10(1) = 0.
    Thus we remove that component completely and only compute the remaining MSE component.
    """
    return -10. * K.log(K.mean(K.square(y_pred - y_true))) / K.log(10.)

Vorerst nur ein sehr einfacher Autoencoder

In [ ]:
from keras.layers import Conv2D,Conv2DTranspose,Input
from keras.models import Model


inp = Input(shape=(tile_size,tile_size,3))

conv1 = Conv2D(64,(3,3), activation='relu',strides=(2,2),padding='same')(inp)
deconv1 = Conv2DTranspose(64,(3,3),activation='relu', strides=(2,2), padding='same')(conv1)
out = Conv2DTranspose(3,(5,5),activation='linear',padding='same')(deconv1)

model = Model(inputs=inp, outputs=out)  
model.compile(loss=keras.losses.mean_squared_error,
             optimizer=keras.optimizers.Adam(),
             metrics=[PSNRLoss])

model.summary()

In [ ]:
%%time
from keras.callbacks import EarlyStopping

early_stopping = EarlyStopping(monitor='loss', patience=5)
hist =  model.fit(train_a, train_b,
                    batch_size=32,
                    epochs=200,
                    shuffle=True,
                    validation_split=0.9,
                    callbacks=[early_stopping],
                    verbose=2)

In [ ]:
import helper
helper.plot_model(hist,size=(16,6))

x = max(hist.history['PSNRLoss'])
print(f"maximum PSNRLoss: {x:.4f}")
x = min(hist.history['loss'])
print(f"minimum loss: {x:f}")

## Testlauf
(ist natürlich verfälscht da hier ein bild aus dem trainingsdatensatz verwendet wird)

In [ ]:
i = 12

plt.figure(figsize=(16,16*3))

#original bild anzeigen
plt.subplot(1,3,1)
plt.imshow(train_b[i],interpolation="nearest")
plt.text(16,-2,"Original",fontsize=20,horizontalalignment='center')

#demosicing durch den autoencoder
plt.subplot(1,3,2)
decoded = model.predict(train_a[i:i+1])
decoded = np.clip(decoded,0,1)
plt.imshow(decoded[0])
plt.text(16,-2,"ML-Demosaic",fontsize=20,horizontalalignment='center')

#opencv demosaicing algorithmus zum vergleich (irgendwas stimmt hier noch nicht)
plt.subplot(1,3,3)
import cv2
(h,w,_) = train_a[i].shape
# rgb bayer bild in graustufen umwandeln
a = np.zeros((h,w))
for x in range(h):
    for y in range(w):
        a[x,y] = sum(train_a[i][x,y]) # summe der 3 kanäle nehmen da 2 kanäle immer 0 sind
a = (a * 255).astype("uint8")
b = cv2.cvtColor(a, cv2.COLOR_BAYER_GB2RGB)
plt.imshow(b,interpolation="nearest")
plt.text(16,-2,"OpenCV Demosaicing",fontsize=20,horizontalalignment='center')

plt.show()